# PyTorch vs TensorFlow in Code

Gibhub code to Blogpost on Medium

In [1]:
import numpy as np
import pandas as pd

data_path = "./data/tweets_short.csv"

data = pd.read_csv(data_path, encoding='utf-8')
data[:10]

,sentiments,tweets
0,1,Sex and the city is the best
1,1,@mileycyrus i'll come if you can get me tickets
2,0,trying to change my bg pic but twiter is not l...
3,0,go to my father's house or not?!that is the qu...
4,0,gonna sleep on this day. gotta get new contact...
5,0,@Lelebee idk if mine does
6,1,@flourishes Sounds like a good way to wake up!...
7,0,Giving my father a foot massage. Keeping him c...
8,1,I have the best friends in the whole world. I ...
9,0,I'm watching Hank n Jim's replay at http://sti...


# Dataset and Preprocessing

In [4]:
# loading data from csv file

data_path = './data/tweets.csv'

data = pd.read_csv(data_path, usecols=[0,5], encoding='utf-8', names=['sentiments', 'tweets'])
data = data.sample(frac=.10)  # shuffle the tweets
data.sentiments = [0 if x==0 else 1 for x in data.sentiments]  # 

new_data_path = "./data/tweets_short.csv"
data.to_csv(new_data_path, header=True, index=False, encoding='utf-8')

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

# instantiate and fit tokenizer
tokenizer = Tokenizer(num_words=20000, oov_token='<00v>')
tokenizer.fit_on_texts(data.tweets)

# transform tweets into sequences of integers
sequences = tokenizer.texts_to_sequences(data.tweets)

# pad sequences so that they have uniform lenth
padded = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=42)
assert(padded.shape==(40000,42))

In [6]:
seq = padded
labels = np.array(data.sentiments)

# TensorFlow 2.0

## 1. Subclassing API

In [9]:
class Subclass_Model(tf.keras.Model):
    
    def __init__(self, embedding_dim=25):
        
        super(Subclass_Model, self).__init__()
        self.embedding_layer = tf.keras.layers.Embedding(input_dim=20000,
                                                         output_dim=50,
                                                         input_length=42)
        
        self.pool1D_layer = tf.keras.layers.GlobalAveragePooling1D()
        self.fc_layer =  tf.keras.layers.Dense(1, activation='sigmoid')
        
    def call(self, inputs):
        
        x = self.embedding_layer(inputs)
        x = self.pool1D_layer(x)
        return self.fc_layer(x)
    
model = Subclass_Model()

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

## 2. Functional API

In [10]:
inputs = tf.keras.layers.Input(shape=(42,))
x = tf.keras.layers.Embedding(input_dim=20000,
                              output_dim=50,
                              input_length=42)(inputs)

x = tf.keras.layers.GlobalAveragePooling1D()(x)
outputs = tf.keras.layers.Dense(units=1, activation='sigmoid')(x)

model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 42)]              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 42, 50)            1000000   
_________________________________________________________________
global_average_pooling1d_3 ( (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 1,000,051
Trainable params: 1,000,051
Non-trainable params: 0
_________________________________________________________________


## 3. Sequential API

In [12]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Embedding(input_dim=20000,
                                    output_dim=50,
                                    input_length=42))

model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 42, 50)            1000000   
_________________________________________________________________
global_average_pooling1d_5 ( (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 51        
Total params: 1,000,051
Trainable params: 1,000,051
Non-trainable params: 0
_________________________________________________________________


## Training a Keras model

In [13]:
# train the model
model.fit(x=seq,
          y=labels,
          batch_size=32,
          epochs=1,
          verbose=2,
          validation_split=0.2)

Train on 32000 samples, validate on 8000 samples
32000/32000 - 22s - loss: 0.6332 - accuracy: 0.6731 - val_loss: 0.5635 - val_accuracy: 0.7383


In [ ]:
# does not work here :(
# tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=False, show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96)

# PyTorch

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

In [15]:
class MyDataset(Dataset):
    
    def __init__(self, tweets, sentiments):
        self.tweets = tweets
        self.sentiments = sentiments
        
    def __getitem__(self, index):
        
        sample = {"tweets":torch.LongTensor(self.tweets[index,:]),
                  "sentiments":self.sentiments[index]}
        
        return sample
        
    def __len__(self):
        return self.tweets.shape[0]
    
    
tweets_dataset = MyDataset(seq, labels.astype(float))
dataloader = DataLoader(tweets_dataset,
                        batch_size=32,
                        shuffle=True)

## 1. Subclassing

In [16]:
class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=20000,
                                            embedding_dim=50)
        self.pooling_layer = nn.AvgPool1d(kernel_size=50)
        self.fc_layer = nn.Linear(in_features=42, out_features=1)
        
    
    def forward(self, inputs):
        
        x = self.embedding_layer(inputs)
        x = self.pooling_layer(x).view(32, 42)
        
        return torch.sigmoid(self.fc_layer(x))
    
model = Model()

## 2. Sequential

In [17]:
model = nn.Sequential(
    nn.Embedding(num_embeddings=20000, embedding_dim=50),
    nn.AvgPool1d(kernel_size=50),
    nn.Flatten(start_dim=1),
    nn.Linear(in_features=42, out_features=1),
    nn.Sigmoid()
)

## Training a PyTorch Model

In [18]:
# PyTorch training loop

#define the loss fn and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# empty list to track batch losses
batch_losses = []

#train the neural network for 5 epochs
for epoch in range(1):

    #reset iterator
    dataiter = iter(dataloader)
    
    #iterate over dataset
    for batch in dataiter:
                
        #reset gradients
        optimizer.zero_grad()
        
        #forward propagation through the network
        out = model(batch["tweets"])
        
        #print(out.shape, batch["sentiments"].shape)
        #calculate the loss
        loss = criterion(out, batch["sentiments"])
        
        #track batch loss
        batch_losses.append(loss.item())
        
        #backpropagation
        loss.backward()
        
        #update the parameters
        optimizer.step()

c:\users\andreas stöffelbauer\appdata\local\programs\python\python37\lib\site-packages\torch\nn\modules\loss.py:498: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
